# №3 - Сomposite quadrature forms.

#### Short rewiew of this task.
1. Achieve by hand accuracy _1e-6_. Compute error by Runge's rule.
2. Calculate $h_{opt}$ using error achieved by Runge's rule. Calculate Integral.
3. Use Richardson's Method. Find **`r`** which gives error less than _1e-6_.
4. Aitken process: Find _m_.
   
Let's look at 2 cases:
* We know the value of `J`.
* The opposite case.

### Importing neccecary modules.

In [2]:
import numpy as np
import scipy.integrate as integrate
import sympy as sp

Set our functions and parameters.

In [3]:
f=lambda x: 4*np.cos(0.5*x)*np.exp(-5/4*x) + 2*np.sin(4.5*x)*np.exp(x/8) + 2
a=1.3; b=2.2; alpha=0; beta=5/6; points=np.array([a, (a+b)/2, b]); n=10
p=lambda x, j: (x-a)**(-alpha) * (b-x)**(-beta)*x**j

### Implement CQF.

Make Newton-Cotes method available for _n=3_ points.

In [74]:
def Neuton_Cotes(a, b, n=3):  # Default n is 3
    x_n = np.linspace(a, b, n)  # Split the segment into n parts
    M = np.array([integrate.quad(p, a, b, args = (j))[0] for j in range(n)])
    X = np.array([[x**i for x in x_n] for i in range(n)], dtype=np.float64)
    A = np.linalg.solve(X, M)
    return np.sum([A[j]*f(x_n[j]) for j in range(n)]);

Here our absolute error.

In [75]:
f_p = lambda x: (4*np.cos(0.5*x)*np.exp(-5/4*x) + 2*np.sin(4.5*x)*np.exp(x/8) + 2) * (b-x)**(-beta)
print('Absolute error is', np.abs(Neuton_Cotes(a, b) - integrate.quad(f_p, a, b)[0]))

Absolute error is 0.5894857656435128


Composite quadrature forms.

In [76]:
def cqf(a, b, points):
    cqf = 0
    for i in range(points.size-1):
        cqf += Neuton_Cotes(points[i], points[i+1])
    return cqf

$$ First \space case \space (We \space know \space our \space "J").  $$

### №1, 2, 4 from rewiew above.

In [77]:
def Aitken(L = 2, h1 = 15):
    h2 = h1 * L
    h3 = h2 * L
    S1 = cqf(a, b, np.linspace(a, b, h1))
    S2 = cqf(a, b, np.linspace(a, b, h2))
    S3 = cqf(a, b, np.linspace(a, b, h3))
    m = -np.log((S3 - S2)/(S2 - S1))/np.log(L)
    print(f'm = {m}')
    return m 

In [84]:
def Runge():
    step = 0
    eps = 1e-6
    error = 1
    n = 2
    m = Aitken()
    L = 2
    while True:
        S1, S2 = cqf(a, b, np.linspace(a, b, n*L)), cqf(a, b, np.linspace(a, b, n))
        error = (S2 - S1)/(1-L**(-m))
        print(f'Step {step}, n = {n}, error is {error}')
        if error < eps:
            break
        n *= L
        step += 1

    h_opt = (b-a)/n*((eps/abs(error)))**(1/m)
    n_opt = int((b-a)//h_opt)
    print(f'Optimal step is {h_opt}. So it\'s mean we shoud split our interval into {n_opt} parts.')
    print(f'The value of quadrature summ using optimal step is {cqf(a, b, np.linspace(a, b, n_opt))}, error is {abs(cqf(a, b, np.linspace(a, b, n_opt)) - integrate.quad(f_p, a, b)[0])}')

In [85]:
Runge()

m = 3.448920585485157
Step 0, n = 2, error is 0.5994729976334708
Step 1, n = 4, error is 0.045777205442445866
Step 2, n = 8, error is 0.003327368693765251
Step 3, n = 16, error is 0.0002977926854778731
Step 4, n = 32, error is 3.0828863349359347e-05
Step 5, n = 64, error is 2.156989692167671e-06
Step 6, n = 128, error is 3.7908569266993734e-07
Optimal step is 0.009314849845401534. So it's mean we shoud split our interval into 96 parts.
The value of quadrature summ using optimal step is 10.839546284415842, error is 1.1543401328140135e-06


/home/loulaan/anaconda3/lib/python3.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)


### №3. Richardson's Method

In [86]:
# %%time
r = 2
m = 2
n = 10
L = 2
eps = 1e-6
while True:
    h = np.array([n*L**i for i in range(r+1)])
    S = np.array([-cqf(a, b, np.linspace(a, b, h[i])) for i in range(h.size)], dtype=np.float64)
    H = np.array([[(b-a)/h[i]**(m+j) for j in range(h.size)] for i in range(h.size)])
    H[:, -1] = -1
    C = np.linalg.solve(H, S)
    R = abs(C[:-1] @ H[-1, :-1])
    print(f'Error is {R}, Quadrature summ is {C[-1]}, step is {(b-a)/h[-1]}')
    if R < eps:
        print(f'Last r is {r}')
        break
    r += 1

Error is 1.6091221131109347e-06, Quadrature summ is 10.839558679281112, step is 0.022500000000000003
Error is 1.0049376546091009e-06, Quadrature summ is 10.839547163669776, step is 0.011250000000000001
Error is 1.4450519371047796e-06, Quadrature summ is 10.839550731257031, step is 0.005625000000000001
Error is 2.3952637378578345e-06, Quadrature summ is 10.839543785270703, step is 0.0028125000000000003


/home/loulaan/anaconda3/lib/python3.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)


Error is 8.710114149869814e-08, Quadrature summ is 10.83954581907276, step is 0.0014062500000000002
Last r is 6


In [104]:
h_opt = 0.0014062500000000002*(eps/R)**(1/m)
h_opt

0.00476486512698193

In [102]:
abs(C[-1] - cqf(a, b, np.linspace(a, b, (b-a)/h_opt)))

/home/loulaan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.
/home/loulaan/anaconda3/lib/python3.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)


5.6883222701031855e-06